In [2]:
# get all imports out of the way
import tensorneat as tn
import jax.numpy as jnp
import pandas as pd

In [19]:
# drop nans split by symbol and join on date
file_path = "hist_data/stocks/sp500_stocks.csv"  # Update this path if needed
df = pd.read_csv(file_path)

df = df.dropna()

symbols = df['Symbol'].unique()
symbol_dfs = {symbol: df[df['Symbol'] == symbol] for symbol in symbols}

merged_df = symbol_dfs[symbols[0]].set_index('Date')

for symbol in symbols[1:]:
    symbol_data = symbol_dfs[symbol].set_index('Date')
    if (len(symbol_data) == 3768):
        merged_df = merged_df.join(symbol_data, how='inner', rsuffix=f"_{symbol}")

tensorneat_input = merged_df.to_numpy()

print(tensorneat_input.shape)

(3768, 1050)


In [20]:
print(merged_df.head())

           Symbol  Adj Close     Close      High       Low      Open  \
Date                                                                   
2010-01-04    AOS   5.937266  7.435000  7.480000  7.261667  7.295000   
2010-01-05    AOS   5.861404  7.340000  7.431667  7.308333  7.431667   
2010-01-06    AOS   5.864068  7.343333  7.405000  7.301667  7.335000   
2010-01-07    AOS   5.881369  7.365000  7.425000  7.311667  7.356667   
2010-01-08    AOS   5.967879  7.473333  7.485000  7.311667  7.331667   

               Volume Symbol_ABT  Adj Close_ABT  Close_ABT  ...    Low_WMB  \
Date                                                        ...              
2010-01-04  1104600.0        ABT      18.763718  26.129908  ...  17.445280   
2010-01-05  1207200.0        ABT      18.612118  25.918797  ...  17.534952   
2010-01-06   663000.0        ABT      18.715481  26.062737  ...  17.771360   
2010-01-07   564000.0        ABT      18.870522  26.278646  ...  18.072985   
2010-01-08   504600.0      

In [ ]:
# create a target array

close_columns = [col for col in merged_df.columns if col.split('_')[0] == 'Close']

# features and target prep
'''
feature_df = merged_df[close_columns].copy()
for c in close_columns:
    feature_df[f'{c}_ma5'] = feature_df[c].rolling(window=5).mean()
    feature_df[f'{c}_ma10'] = feature_df[c].rolling(window=10).mean()
    feature_df[f'{c}_ma20'] = feature_df[c].rolling(window=20).mean()
    feature_df[f'{c}_avg_change_13'] = feature_df[c].pct_change().rolling(window=13).mean()
    feature_df[f'{c}_stddev_13'] = feature_df[c].pct_change().rolling(window=13).std()
print(feature_df.values.shape)
'''

# shift target and adjust features accordingly
target = merged_df[close_columns].pct_change().shift(-1).values[:-1,:]
features = merged_df[close_columns].pct_change().values[1:,:]
print(target[-1])

[ 0.00409116  0.01610038  0.04501611 -0.0030364   0.01986408  0.01718508
  0.01604073  0.00729995  0.01900508  0.01209675  0.00814169  0.01165987
  0.02247875  0.01436816  0.0079752   0.02600378  0.01306534  0.02076126
  0.01720051  0.00410805  0.01583561  0.01270647  0.00327085  0.00615833
  0.0287785   0.01702146  0.00047481  0.06433673  0.01573387  0.01820082
  0.01204401  0.01544333  0.01125408  0.00188873  0.00722064  0.02192512
  0.01077803  0.00255451  0.00531237  0.0229096   0.01373552  0.05565815
  0.00995424  0.02298103  0.03396944 -0.00341414  0.0017466   0.01726898
  0.01752603  0.01921585  0.02040288 -0.00060854  0.00797187  0.01880466
  0.0061308   0.02676645  0.01702448 -0.00455149  0.02976421  0.00734793
 -0.0126805   0.01067136  0.02163391 -0.00054369  0.03701384  0.02467057
  0.02603908  0.01621336 -0.00624905  0.01311265  0.02251078  0.01261788
  0.00316657  0.03233135  0.0100828   0.02654338  0.03901625  0.01362199
 -0.00232726  0.01132988  0.00099257  0.03424518  0

In [27]:
import jax.numpy as jnp
train_test_split = 0.8

sample_count = target.shape[0]
print(f"Sample Count: {sample_count}")
train_size = int(sample_count * train_test_split)
print(f"Train Size: {train_size}")
TEST_INPUTS = features[train_size:, :]
TEST_RETURNS = target[train_size:, :]
INPUTS = features[:train_size, :]
RETURNS = target[:train_size, :]

print(f"Shape of INPUTS: {INPUTS.shape}")
print(f"Shape of RETURNS: {RETURNS.shape}")

Sample Count: 3767
Train Size: 3013
Shape of INPUTS: (3013, 150)
Shape of RETURNS: (3013, 150)


In [ ]:
import jax, jax.numpy as jnp
from tensorneat.problem.base import BaseProblem
import tensorneat.algorithm.neat as neat_algorithm
import matplotlib.pyplot as plt
import random
EPISODE_LEN = 34

class TradingProblem(BaseProblem):
  jitable = True
  episode_len = EPISODE_LEN
  pop_size = 100
  
  def __init__(self):
      super().__init__()
      self.pop_counter = 0
      self.start_idx = random.randint(0, INPUTS.shape[0] - self.episode_len)
      self.end_idx = self.start_idx + self.episode_len

  @property
  def input_shape(self):
    return (INPUTS.shape[1],)

  @property
  def output_shape(self):
    return (1,)

  def evaluate(self, state, randkey, act_func, params):
    actions = jax.vmap(act_func, in_axes=(None, None, 0))(state, params, INPUTS)
    actions_buy_sell = jnp.where(actions > 0.0, 1, 0)
    reward = actions_buy_sell * RETURNS
    return jnp.mean(reward)

  def show(self, state, randkey, act_func, params, *args, **kwargs):
    # get actions as a host numpy array
    actions = jax.vmap(act_func, in_axes=(None, None, 0))(state, params, TEST_INPUTS)
    actions_buy_sell = jnp.where(actions > 0.0, 1, 0)

    # move data to host numpy
    from jax import device_get
    import numpy as np
    actions_np = np.asarray(device_get(actions_buy_sell)).ravel()
    returns_np = np.asarray(device_get(TEST_RETURNS)).ravel()
    print("latest action", actions_np[-1])
    perf = 100.0
    perf_hist = []
    for i in range(len(actions_np)):
      perf += perf * float(returns_np[i] * actions_np[i])
      perf_hist.append(float(perf))

    # plot using numpy list of floats
    import matplotlib.pyplot as plt
    plt.plot(perf_hist)
    print(perf_hist[0], perf_hist[-1])
    plt.show()

# Assuming INPUTS and RETURNS are defined from previous cells
trading_problem = TradingProblem()



In [ ]:
from tensorneat.pipeline import Pipeline
from tensorneat.algorithm.neat import NEAT
from tensorneat.genome import DefaultGenome, BiasNode, DefaultConn, DefaultMutation
from tensorneat.problem.func_fit import CustomFuncFit
from tensorneat.common import ACT, AGG
from tensorneat import algorithm

# Construct the pipeline and run
pipeline = Pipeline(
    algorithm=NEAT(
        pop_size=21,
        species_size=3,
        survival_threshold=0.1,
        compatibility_threshold=0.8,
        genome=DefaultGenome(
            max_nodes=100,
            max_conns=1500,
            num_inputs=features.shape[1],
            num_outputs=target.shape[1],
            init_hidden_layers=(2,),
            node_gene=BiasNode(
                bias_init_std=0.1,
                bias_mutate_power=0.05,
                bias_mutate_rate=0.01,
                bias_replace_rate=0.0,
                activation_options=ACT.tanh,
                aggregation_options=AGG.sum,
            ),
            conn_gene=DefaultConn(
                weight_init_mean=0.0,
                weight_init_std=0.1,
                weight_mutate_power=0.05,
                weight_replace_rate=0.0,
                weight_mutate_rate=0.001,
            ),
            output_transform=ACT.tanh,
        ),
    ),
    problem=TradingProblem(),
    generation_limit=5,
    fitness_target=.01,
    seed=42,
    is_save=True,
    save_dir="./model_archive/tensorneat_checkpoints"
)

# initialize state
state = pipeline.setup()
# run until terminate
state, best = pipeline.auto_run(state)
# show result
pipeline.show(state, best)
state.save("./model_archive/tensorneat_checkpoints/evolving_state.pkl")